# Preprocessing Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as skp

## First Formatting of the Data

In [ ]:
train_data = pd.read_csv("data/UNSW_NB15_training-set.csv")
test_data = pd.read_csv("data/UNSW_NB15_testing-set.csv")

## Some Small Exploration of the Data

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

## Removing Unneeded Columns

In [ ]:
train_data = train_data.drop(columns=["id", "attack_cat"])
test_data = test_data.drop(columns=["id", "attack_cat"])

## Converting Data to Numerical Values

In [ ]:
def encode_categoricals(data):
    categorical_cols = [c for c in data if data[c].dtype == "object"]
    data[categorical_cols] = skp.OrdinalEncoder().fit_transform(data[categorical_cols])
    
encode_categoricals(train_data)
encode_categoricals(test_data)

## Normalization

In [ ]:
def byterize(x):
    return 2**np.ceil(np.log2(x))

def min_max_scale(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val)

def normalize(train_data, test_data):
    for col in train_data:
        min_val = np.floor(train_data[col].min())
        max_val = byterize(train_data[col].max())
        test_data[col] = test_data[col].clip(upper=max_val)
        train_data[col] = min_max_scale(train_data[col], min_val, max_val)
        test_data[col] = min_max_scale(test_data[col], min_val, max_val)
        
normalize(train_data, test_data)

## Final Formatting

In [ ]:
def extract_X_Y(data):
    Y = data.label.to_numpy()
    X = data.drop(columns="label").to_numpy()
    return X, Y

X_train, Y_train = extract_X_Y(train_data)
X_test, Y_test = extract_X_Y(test_data)

# Supervised Machine Learning

In [ ]:
import tensorflow as tf

def create_model(sample_shape):
    inputs = tf.keras.Input(sample_shape)
    x = tf.keras.layers.Dense(100, activation="relu")(inputs)
    x = tf.keras.layers.Dense(50, activation="relu")(x)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="binary_crossentropy",
        metrics=["binary_accuracy", "AUC"]
    )
    return model

model = create_model(X_train.shape[1:])
model.fit(x=X_train, y=Y_train, batch_size=128, epochs=3)
loss, acc, auc = model.evaluate(X_test, Y_test)
print(f"LOSS: {loss}, Accuracy: {acc:%}, AUC: {auc}")

# Unsupervised Machine Learning

In [ ]:
from sklearn import cluster as skc
from sklearn import metrics as skm

model = skc.KMeans(n_clusters=2, n_init='auto')
model.fit(X_train)

attack_idx = Y_train == 1
attack_cluster = model.predict(X_train[attack_idx]).mean().round()

In [ ]:
def predict(model, X_train, Y_train, X_test, Y_test):
    attack_idx = Y_train == 1
    attack_cluster = model.predict(X_train[attack_idx]).mean().round()
    return 1 - model.predict(X_test) if attack_cluster == 0 else model.predict(X_test)

preds = predict(model, X_train, Y_train, X_test, Y_test)

In [ ]:
loss = skm.log_loss(Y_test, preds)
acc = skm.accuracy_score(Y_test, preds)
mcc = skm.matthews_corrcoef(Y_test, preds)
print(f"Loss: {loss}, Accuracy: {acc:%}, MCC: {mcc}")